# Intro

***Notebook in progress.***

This notebook creates some basic functions for generating weird spells for Dungeons and Dragons. The basic functions build throughout the notebook to a larger generator with much more modularity at the end.

## Imports

In [1]:
import numpy as np
from numpy.random import choice as nrc
import pandas as pd

# Making Things Up

Here we make some lists of stuff to randomly pull from. Schools of magic, types of damage, different fun groups of words to generate from, etc.

In [269]:
# Top block definitions
levels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
schools = ['abjuration', 'conjuration', 'divination', 'enchantment',
           'evocation', 'illusion', 'necromancy', 'psionics', 'transmutation']
cast_times = ['action', 'bonus action', 'reaction', 'one minute', 'one hour']
durations = ['instantaneous', 'one minute', 'one round', 'one hour', 'permanent']
ranges = ['self', 'point', 'touch']

# Mechanical
dice = ['d4', 'd6', 'd8', 'd10', 'd12', 'd20']
stats = ['charisma', 'constitution', 'dexterity', 'intelligence', 'strength',
         'wisdom']
shapes = ['ball', 'beam', 'cone', 'cube', 'cylinder', 'disc', 'donut',
          'line', 'ray', 'ring', 'sphere', 'wall']

In [3]:
# The damage types ['earth', 'water', 'wind'] are used in my home game
# and have been added to this list.
damage_types = ['acid', 'bludgeoning', 'cold', 'earth', 'fire', 'force',
                'lightning', 'necrotic', 'piercing', 'poison', 'psychic',
                'radiant', 'slashing', 'thunder', 'water', 'wind']

In [4]:
# Verbs based on school of spell
evoc_verbs = ['blast', 'call forth', 'conjure', 'create', 'demand', 'evoke',
              'launch', 'shoot', 'summon']
trans_verbs = ['change', 'switch', 'transmute', 'turn']

In [5]:
# Fun lists to pull words from
body_parts = ['ankle', 'arm', 'artery', 'beard', 'belly', 'body', 'bone',
              'bowels', 'brain', 'bladder', 'chin', 'calf', 'chest',
              'cheek', 'clavicle', 'ear', 'eye', 'eyebrow', 'eyelid',
              'face', 'finger', 'fingernail', 'foot', 'groin', 'gums',
              'hair', 'hand', 'head', 'heart', 'horns', 'iris', 'intestines',
              'jaw', 'jowels', 'kneecap', 'leg', 'liver', 'lungs',
              'lymph node', 'molars', 'mouth', 'muscle', 'ovary', 'palm',
              'pelvis', 'red blood cells', 'retina', 'ribs',
              'scalp', 'shin', 'skeleton', 'skull', 'spine', 'spleen',
              'stomach', 'teeth', 'thorax', 'throat', 'tongue', 'tonsils',
              'uterus', 'white blood cells']

## Importing Spell Components

This kindly [Reddit user](https://www.reddit.com/user/hillermylife) put together a [spreadsheet](https://docs.google.com/spreadsheets/d/1KSibOeWub0_f79GYSnMu7om8kWwog1ob8dRY9LLoDAE/edit#gid=0) with all the D&D 5E spell components on it. I'm importing that from a `.csv` with Pandas and storing the components as a list.

In [6]:
df = pd.read_csv('data/spell_components.csv')
df

,Material,Value,Spell,Consumed?,Source,Pg.,Lvl.,Notes,School,Artificer,Bard,Cleric,Druid,Paladin,Ranger,Sorcerer,Warlock,Wizard
0,"-If you find this spreadsheet valuable, I hope...",-Items in orange and italic are suggested pric...,-You seem like a person who likes materials. I...,NaN,-,-,-1.0,"-If you have any suggestions, feel free to add...",-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[An Item Distasteful to the Target],-,Banishment,N,PHB,217,4.0,"Obviously, this is highly variable.",Abjuration,False,False,True,False,True,False,True,True,True
2,"Adamantine, small piece",500 gp,Invulnerability,Y,XGtE,160,9.0,NaN,Illusion,False,True,False,False,False,False,False,True,True
3,Adder's stomach,8 gp (suggested),Melf's Acid Arrow,N,PHB,259,2.0,NaN,Evocation,False,False,False,False,False,False,False,False,True
4,Agate,"1,000 gp",Awaken,Y,PHB,216,5.0,NaN,Transmutation,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,Wisp of Smoke,-,Gaseous Form,Y,PHB,244,3.0,NaN,Transmutation,False,False,False,False,False,False,True,True,True
357,Wisp of Smoke,-,True Polymorph,Y,PHB,266,9.0,NaN,Transmutation,False,True,False,False,False,False,False,True,True
358,Wood,1 sp (suggested),Unseen Servant,N,PHB,284,1.0,NaN,Conjuration,False,True,False,False,False,False,False,True,True
359,Wychwood,5 gp (suggested),Dancing Lights,N,PHB,230,0.0,NaN,Evocation,True,True,False,False,False,False,True,False,True


In [7]:
components = list(df['Material'][2:361].drop_duplicates().str.lower())
# components

# Smaller Functions

## `spell_name()` Function

Most basic version of spell generation. These funny titles are intended to serve as a jumping off point for creation.

In [8]:
def spell_name():
    rand_school = nrc(schools)
    rand_dmg = nrc(damage_types)
    rand_part = nrc(body_parts)
    
    determinant = np.random.randint(2)
    if determinant == 0:
        name = f"{rand_school} Spell: {rand_part} Of {rand_dmg}".title()
    else:
        name = f"{rand_school} Spell: {rand_dmg} {rand_part}".title()
    return name

And some examples.

In [50]:
spell_name()

'Illusion Spell: Jowels Of Psychic'

In [53]:
spell_name()

'Divination Spell: Kneecap Of Force'

In [54]:
spell_name()

'Evocation Spell: Bowels Of Bludgeoning'

## `jumble_name()` Function

This function creates weirder names, sometimes nonsensical. Intended to do the same as above, but with less logic to it.

In [9]:
def jumble_name():
    rand_school = nrc(schools)
    rand_dmg = nrc(damage_types)
    rand_part = nrc(body_parts)
    filler = np.random.permutation([rand_school, rand_dmg, rand_part])
    name = ("%s %s of %s" % (filler[0], filler[1], filler[2])).title()
    return name

And some examples.

In [37]:
jumble_name()

'Chin Wind Of Evocation'

In [48]:
jumble_name()

'Lightning Skeleton Of Enchantment'

In [50]:
jumble_name()

'Abjuration Arm Of Earth'

In [63]:
jumble_name()

'Force Psionics Of Beard'

In [67]:
jumble_name()

'Radiant Illusion Of Molars'

The intention here is that the new spell names given cause you to reconsider traditional game motifs. In the final example above, `'Radiant Illusion Of Molars`, we're given two words that normally come up in D&D spells--radiant and illusion.
>One is a type of damage--radiant tends to be angelic and defeat undead, demons, and the like--and the other is a school of magic--illusions, fake images and sounds, etc.

But now it's like... A spell where I create the illusion that I have really *radiant* teeth? *Shiny* chompers? How can I invent a usage for that in an RPG?

## `simple_desc()` Function

This function prints out a basic box of "flavor text," i.e. the kind of thing that explains what's *going on* when the spell happens. It's the difference between knowing that the spell Fireball makes a ball of fire, and knowing that it explodes from a single point, wraps around corners, and can burn pieces of terrain, furniture, and items.

In [10]:
def simple_desc():
    rand_dmg = nrc(damage_types)
    rand_school = nrc(schools)
    rand_part = nrc(body_parts)
    rand_verb = nrc(evoc_verbs)
    rand_shape = nrc(shapes)
    rand_stat = nrc(stats)
    rand_num = np.random.randint(2,30)
    rand_dice = nrc(dice)
    aoe = 5 * np.random.randint(1,20)
    
    title = f"{rand_school} Spell: {rand_part} Of {rand_dmg}".title()
    body = f"You {rand_verb} a {rand_shape} of {rand_dmg} from your {rand_part}. All creatures within {aoe}ft. of you must make a {rand_stat} saving throw or take {rand_num}{rand_dice} {rand_dmg} damage."
    print(title, "\n\n", body)

In [47]:
simple_desc()

Abjuration Spell: Bowels Of Fire 

 You conjure a cone of fire from your bowels. All creatures within 30ft. of you must make a wisdom saving throw or take 6d6 fire damage.


In [48]:
simple_desc()

Psionics Spell: Spleen Of Wind 

 You call forth a ring of wind from your spleen. All creatures within 85ft. of you must make a strength saving throw or take 3d20 wind damage.


In [50]:
simple_desc()

Divination Spell: Beard Of Force 

 You evoke a beam of force from your beard. All creatures within 75ft. of you must make a constitution saving throw or take 26d6 force damage.


## `trasmutation()` Function

In [11]:
def transmutation():
    trans_verb = nrc(trans_verbs)
    thing_1 = nrc(components)
    thing_2 = nrc(components)
    print(f"You {trans_verb} a {thing_1} into a {thing_2}.")

In [50]:
transmutation()

You turn a pickled octopus tentacle into a shamrock.


In [52]:
transmutation()

You transmute a sponge into a pearl.


In [57]:
transmutation()

You turn a clay pot of brackish water into a statue of the caster, carved from ivory and decorated with gems.


# Bringing Bits Together

***Very unfinished.***

I now make a function that creates a more in-depth spell description, with the option of saving it as a `.txt` file locally.

In [172]:
# # # THIS WILL PROB BE SCRAPPED - TOO MESSY # # #

def full_spell(school = None, filepath = None):
# Spell school
    if not school:
        school = nrc(schools)
# Spell level
    level = nrc(levels)
# Spell casting time, duration, and concentration
    casting = nrc(cast_times, p = [0.75,0.1,0.05,0.05,0.05])
    duration = nrc(durations, p = [0.6,0.2,0.1,0.05,0.05])
    concentration = nrc([True, False])
# Spell range, area of effect, shape
    range_ = nrc(ranges)
    aoe = nrc([True, False])
    shape = None
    if range_ == 'point':
        range_ = 5 * np.random.randint(1, 40)
        if aoe:
            aoe = 5 * np.random.randint(1,10)
            shape = nrc(shapes)
# Assembling a full text block
    text = ""
# Saving finished spells locally      
    if filepath:
        # RUDIMENTARY SAVING AS PROOF OF CONCEPT
        # FLESH OUT LATER
        file = open(filepath, 'a')
        file.write('\n===\n')
        for i in [level, school]:
            file.write(i)
            file.write('\n')
        file.close()
    return level, school, range_, aoe, shape, duration, casting, concentration

In [174]:
full_spell(filepath = 'data/generated_spells.txt')

('2', 'necromancy', 'self', True, None, 'instantaneous', 'action', True)

## `evocation()` Function

In lieu of the previous function getting a bit hard to wrangle, I'm first making a basic evocation spell template. I'll likely do this for each school of magic, and then combine them into a larger function that just randomly picks a school and goes from there. Having the logic all be interconnected is a bit messy.

In [295]:
def evocation(level = None):
# Assign level and element
    if not level:
        level = nrc(levels)
    element = nrc(damage_types)
# Range, area of effect, and shape
    range_ = nrc(['touch', 'point'])
    aoe = nrc([True, False])
    shape = None
    if range_ == 'point':
        range_ = 5 * np.random.randint(1, 40)
        if aoe:
            aoe = 5 * np.random.randint(1,10)
            shape = nrc(shapes)
# Create a spell name
    part = nrc(body_parts)
    if nrc([True, False]):
        name = f"{part} Of {element}".title()
    else:
        name = f"{element} {part}".title()
# Create a spell decription based on whether or not the spell has an
# area of effect
    verb = nrc(evoc_verbs)
    if shape:
        damage = level * 2 + np.random.randint(3)
        stat = nrc(stats)
        damage_desc = (f"You {verb} a {shape} of {element} from your {part} at a point within range." +
                       f" All creatures within {aoe}ft. must make a {stat} saving throw" +
                       f" or take {damage}d6 {element} damage.")
    else:
        damage = level + 2 + np.random.randint(1, 3)
        damage_desc = (f"You {verb} {element} from your {part} at a creature within range." +
                       f" If it hits, it deals {damage}d10 {element} damage.")
# Return things
    return name, level, range_, aoe, shape, element, damage_desc

Example.

In [304]:
evocation()

('Throat Of Bludgeoning',
 4,
 35,
 20,
 'ball',
 'bludgeoning',
 'You blast a ball of bludgeoning from your throat at a point within range. All creatures within 20ft. must make a intelligence saving throw or take 8d6 bludgeoning damage.')